## Binary Tree Implemntation

In [2]:
from typing import List, Deque, Optional, Union, Any, Dict, Iterator
from dataclasses import dataclass
import math

### Complete Binary Tree

In [3]:
class CBTNode:
    def __init__(self, 
                 val: int, 
                 left: Optional[int] = None,
                 right: Optional[int] = None) -> None:
        self.value = val
        self.left = left
        self.right = right
    
def init_BT(arr: List[int]) -> CBTNode:
    queue = []
    root = CBTNode(arr[0])
    current_node = root
    for item in range(1, len(arr)):
        if not current_node.left:
            current_node.left = CBTNode(item)
            queue.append(current_node.left)
        elif not current_node.right:
            current_node.right = CBTNode(item)
            queue.append(current_node.right)
        if current_node.left and current_node.right:
            current_node = queue.pop(0)
    return root

### Binary Tree Traversal

- Pre-order
- In-order:
- Post-order

In [4]:
def DST_BT(node: CBTNode):
    '''
    Inorder traversal
    '''
    if not node: return
    DST_BT(node.left)
    print(node.value)
    DST_BT(node.right)
    
def BST_BT(node: CBTNode):
    pass

### Best Binary Tree

In [5]:
__all__ = [
    "Node",
    "build",
    "get_index",
    "get_parent",
    "NodeValue",
    "NodeValueList"
]

_ATTR_LEFT = "left"
_ATTR_RIGHT = "right"
_ATTR_VALUE = "value"

_NODE_VAL_TYPES = (float, int, str)
NodeValue = Any
NodeValueList = Union[
    List[Optional[int]],
    List[Optional[float]],
    List[Optional[str]],
    List[int],
    List[float],
    List[str] 
]

@dataclass
class NodeProperties:
    height: int
    size: int
    is_max_heap: bool
    is_min_heap: bool
    is_perfect: bool
    is_complete: bool
    is_full: bool
    leaves_count: int
    min_node_val: NodeValue
    max_node_val: NodeValue
    min_leaf_depth: int
    max_leaf_depth: int

In [98]:
class Node:
    def __init__(self, value: NodeValue, left: Optional["Node"] = None, right: Optional["Node"] = None) -> None:
        self.value = value
        self.left = left
        self.right = right
        
    def __repr__(self) -> str:
        return "Node({})".format(self.value)    
    
        
    def __iter__(self) -> Iterator["Node"]:
        current_nodes: Optional[Node] = [self]
        while len(current_nodes) > 0:
            next_nodes = []
            for node in current_nodes:
                yield node
                if node.left is not None:
                    next_nodes.append(node.left)
                if node.right is not None:
                    next_nodes.append(node.right)
            current_nodes = next_nodes
        
    
    def __len__(self) -> int:
        return sum(1 for _ in iter(self))
    
    def __setattr__(self, attr: str, obj: Any) -> None:
        if attr == _ATTR_LEFT:
            if obj is not None and not isinstance(obj, Node):
                raise Exception("left child must be a Node instance")
        elif attr == _ATTR_RIGHT:
            if obj is not None and not isinstance(obj, Node):
                raise Exception("right child must be a Node instance")
        elif attr == _ATTR_VALUE:
            if not isinstance(obj, _NODE_VAL_TYPES):
                raise Exception("node value must be float/int/str")
        object.__setattr__(self, attr, obj)
    
    def validate(self) -> None:
        '''
        Check if the binary tree is malformed: cyclic reference, node type error, node value error
        '''
        nodes_seen = set() # hash lookup is used in sets, which makes `in` operator more efficient for sets than lists
        has_more_nodes = True
        current_nodes: Optional[Node] = [self]
        node_index = 0
        
        while has_more_nodes:
            next_nodes: Optional[Node] = []
            has_more_nodes = False
            
            for node in current_nodes:
                if node is None:
                    pass
                else:
                    if node in nodes_seen:
                        raise Exception('cyclic reference at Node({}) (level-order index {})'.format(node.value, node_index))    
                    if not isinstance(node, Node):
                        raise Exception('invalid node instance at index {}'.format(node_index))
                    if not isinstance(node.value, _NODE_VAL_TYPES):
                        raise Exception('invalid node value at index {}'.format(node_index))
                    if node.left is not None or node.right is not None:
                        has_more_nodes = True
                        next_nodes.append(node.left)
                        next_nodes.append(node.right)
                    nodes_seen.add(node)
                node_index += 1
            
            current_nodes = next_nodes
                
                
    def equals(self, other: "Node") -> bool:
        stack1: List[Optional[Node]] = [self]
        stack2: List[Optional[Node]] = [other]
        
        while stack1 or stack2:
            node1 = stack1.pop()
            node2 = stack2.pop()
            if node1 is None and node2 is None:
                continue
            elif node1 is None or node2 is None:
                return False
            elif not isinstance(node2, Node):
                return False
            else:
                if node1.value != node2.value:
                    return False
                stack1.append(node1.left)
                stack1.append(node1.right)
                stack2.append(node2.left)
                stack2.append(node2.right)
        return True
    
    @property
    def values(self) -> List[Optional[NodeValue]]:
        '''
        return a list of node values in breadth-first order starting from the root
        '''
        node_values: List[Optional[int]] = []
        current_nodes: List[Optional[Node]] = [self]
        has_more_nodes = True
        
        while has_more_nodes:
            has_more_nodes = False
            next_nodes: List[Optional[Node]] = []
            
            for node in current_nodes:
                if node is None:
                    node_values.append(None)
                    next_nodes.append(None)
                    next_nodes.append(None)
                else:
                    if node.left is not None or node.right is not None:
                        has_more_nodes = True
                    node_values.append(node.value)
                    next_nodes.append(node.left)
                    next_nodes.append(node.right)
            
            current_nodes = next_nodes
            
        while node_values and node_values[-1] is None:
            node_values.pop()
            
        return node_values
        
    @property
    def values2(self) -> List[Optional[NodeValue]]:
        '''
        return a list of node values in more compact representation
        '''
        current_nodes: List[Optional[Node]] = [self]
        has_more_node = True
        node_values: List[Optional[NodeValue]] = [self.value]
        
        while has_more_node:
            next_nodes: List[Optional[Node]] = []
            has_more_node = False
            for node in current_nodes:
                for child in node.left, node.right: 
                    if child is None:
                        node_values.append(None)
                    else:
                        has_more_node = True
                        next_nodes.append(child)
                        node_values.append(child.value)
            current_nodes = next_nodes
                
        while node_values and node_values[-1] is None:
            node_values.pop()
            
        return node_values
        
    
    @property
    def leaves(self) -> List["Node"]:
        current_nodes: List[Optional[Node]] = [self]
        leaf_values: List[Node] = []
        
        while len(current_nodes) > 0:
            next_nodes: List[Node] = []
            for node in current_nodes:
                if node.left is None and node.right is None:
                    leaf_values.append(node)
                    continue
                if node.left is not None:
                    next_nodes.append(node.left)
                if node.right is not None:
                    next_nodes.append(node.right)
            current_nodes = next_nodes
        return leaf_values
                
            
    @property
    def levels(self) -> List[List["Node"]]:
        current_nodes: List[Optional[Node]] = [self]
        levels: List[List[Node]] = []
        
        while len(current_nodes) > 0:
            levels.append(current_nodes)
            next_nodes: Optional[Node] = []
            for node in current_nodes:
                for child in node.left, node.right:
                    if child is not None:
                        next_nodes.append(child)
            current_nodes = next_nodes
            
        return levels
    
    @property
    def height(self) -> int:
        return _get_tree_properties(self).height
    
    @property
    def size(self) -> int:
        return self.__len__()
    
    @property
    def leaves_count(self) -> int:
        return _get_tree_properties(self).leaves_count
    
    @property
    def is_balanced(self) -> bool:
        return _is_balanced(self)
    
    @property
    def is_symmetric(self) -> bool:
        return _is_symmetric(self)
    
    @property
    def is_bst(self) -> bool:
        return _is_bst(self)
    
    @property
    def is_strict(self) -> bool:
        return _get_tree_properties(self).is_strict
    
    @property
    def is_full(self) -> bool:
        return _get_tree_properties(self).is_full
    
    @property
    def is_complete(self) -> bool:
        return _get_tree_properties(self).is_complete
    
    @property 
    def is_min_heap(self) -> bool:
        return _get_tree_properties(self).is_min_heap
    
    @property
    def is_max_heap(self) -> bool:
        return _get_tree_properties(self).is_max_heap
    
    @property 
    def min_node_value(self) -> NodeValue:
        return _get_tree_properties(self).min_node_value
    
    @property 
    def max_node_value(self) -> NodeValue:
        return _get_tree_properties(self).max_node_value
    
    @property
    def min_leaf_depth(self) -> int:
        return _get_tree_properties(self).min_leaf_depth
    
    @property
    def max_leaf_depth(self) -> int:
        return _get_tree_properties(self).max_leaf_depth
    
    @property
    def properties(self) -> Dict[str, Any]:
        properties = _get_tree_properties(self).__dict__.copy()
        properties["is_balanced"] = _is_balanced(self) >= 0
        properties["is_bst"] = _is_bst(self)
        properties["is_symmetric"] = _is_symmetric(self)
        return properties

    
    @property
    def inorder(self) -> List["Node"]:
        stack: List[Node] = []
        res: List[Node] = []
        node: Optional[Node] = self
        
        while node or stack:
            while node:
                stack.append(node)
                node = node.left
            if stack:
                node = stack.pop()
                res.append(node)
                node = node.right
                
        return res
        
    @property
    def preorder(self) -> List["Node"]:
        stack: List[Node] = []
        res: List[Node] = []
        node: Node = self
        
        while stack or node:
            while node:
                res.append(node)
                stack.append(node)
                node = node.left
            if stack:
                node = stack.pop()
                node = node.right
                
        return res
                
    @property
    def postorder(self) -> List["Node"]:
        stack: List[Node] = [self]
        res: List[Node] = []
        
        while stack:
            node = stack.pop()
            if node:
                res.append(node)
                stack.append(node.left)
                stack.append(node.right)
        
        return res[::-1]
        
    
    @property
    def levelorder(self) -> List["Node"]:
        res: List[Node] = []
        stack: List[Node] = [self]
        while stack:
            node = stack.pop(0)
            if node:
                res.append(node)
                stack.append(node.left)
                stack.append(node.right)
        return res


def _is_balanced(root: Node) -> bool:
    if root is None:
        return 0
    left = _is_balanced(root.left)
    if left < 0: return -1
    right = _is_balanced(root.right)
    if right < 0: return -1
    return -1 if abs(left - right) > 1 else max(left, right) + 1


def _is_symmetric(root: Node) -> bool:
    
    def symmetric_helper(left: Node, right: Node) -> bool:
        if left is None and right is None:
            return True
        if left is None or right is None:
            return False
        return left.value == right.value and \
            symmetric_helper(left.right, right.left) and \
            symmetric_helper(left.left, right.right)

    return symmetric_helper(root.left, root.right)
    

def _is_bst(root: Node) -> bool:
    cur: Optional[Node] = root
    pre = None
    stack: List[Node] = []
    
    while cur or stack:
        while cur is not None:
            stack.append(cur)
            cur = cur.left
        if stack:
            node = stack.pop()
            if pre is not None and pre.value >= node.value:
                return False
            pre = node
            cur = node.right
        
    return True
        


def _get_tree_properties(root: Node) -> NodeProperties:
    max_leaf_depth: int = -1
    min_leaf_depth: int = 0
    size: int = 0
    is_full: bool = True
    is_complete: bool = True
    not_full_node_seen: bool = False
    is_descending: bool = True
    is_ascending: bool = True
    leaves_count: int = 0
    max_node_val: int = 0
    min_node_val: int = 0
    current_nodes: List[Optional[Node]] = [root]
    
    while len(current_nodes):
        max_leaf_depth += 1
        next_nodes: List[Node] = []
        
        for node in current_nodes:
            size += 1
            val = node.value
            min_node_val = min(min_node_val, val)
            max_node_val = max(max_node_val, val)
            
            if node.left is None and node.right is None:
                leaves_count += 1
                if min_leaf_depth == 0:
                    min_leaf_depth = max_leaf_depth
            
            if node.left is not None:
                next_nodes.append(node.left)
                if node.left.value > val:
                    is_descending = False
                elif node.left.value < val:
                    is_ascending = False
                is_complete = not not_full_node_seen
            else:
                not_full_node_seen = True
            
            if node.right is not None:
                next_nodes.append(node.right)
                if node.right.value > val:
                    is_descending = False
                elif node.right.value < val:
                    is_ascending = False
                is_complete = not not_full_node_seen
            else:
                not_full_node_seen = True
                
            is_full &= (node.left is None) == (node.right is None)
        
        current_nodes = next_nodes
    
    return NodeProperties(
        height=max_leaf_depth,
        size=size,
        is_max_heap=is_complete and is_descending,
        is_min_heap=is_complete and is_ascending,
        is_full=is_full,
        is_complete=is_complete,
        is_perfect=(leaves_count == 2**max_leaf_depth),
        max_node_val=max_node_val,
        min_node_val=min_node_val,
        leaves_count=leaves_count,
        max_leaf_depth=max_leaf_depth,
        min_leaf_depth=min_leaf_depth
    )
    

def build(values: NodeValueList) -> Optional[Node]:
    '''
    breadth-first order starting from the root (current node). 
    If a node is at index i, its left child is always at 2i + 1, right child at 2i + 2, 
    and parent at floor((i - 1) / 2). 
    '''
    nodes = [Node(item) for item in values]
    for index in range(1, len(values)):
        node = nodes[index]
        if node is not None:
            parent_index = (index - 1) // 2
            parent = nodes[parent_index]
            if parent is None:
                raise Exception("parent node missing at {}".format(parent_index))
            setattr(parent, _ATTR_LEFT if index % 2 else _ATTR_RIGHT, node)
    return nodes[0] if nodes else None

def build2(values: NodeValueList) -> Optional[Node]:
    '''
    a slightly different representation which associates two adjacent child values with the first parent value 
    that has not been associated yet. 
    This representation does not provide the same indexing properties where if a node is at index i, 
    its left child is always at 2i + 1, right child at 2i + 2, and parent at floor((i - 1) / 2).
    but it allows for more compact lists as it does not hold "None"s between nodes in each level.
    ----
    root = build2([2, 5, None, 3, None, 1, 4])
                2
               /
            __5
           /
          3
         / \\
        1   4
    '''
    queue: Deque[Node] = []
    root: Optional[Node] = None
    
    if values:
        root = Node(values[0])
        queue.append(root)
    
    index = 1
    while index < len(values):
        node = queue.pop(0)
        if values[index] is not None:
            node.left = Node(values[index])
            queue.append(node.left)
        index += 1
        if index < len(values) and values[index] is not None:
            node.right = Node(values[index])
            queue.append(node.right)
        index += 1
    
    return root
    
def get_index(root: Node, descendent: Node) -> int:
    '''
    get_index(root, root.left.right) --> 4
    '''
    if not isinstance(root, Node):
        raise Exception('the given root is not a node instance')
    if not isinstance(descendent, Node):
        raise Exception('the given descendent is not a node instance')
    index = 0
    current_nodes: List[Optional[Node]] = []
    has_more_node = True
    while has_more_node:
        has_more_node = False
        next_nodes: List[Optional[Node]] = []
        for node in current_nodes:
            if node is not None and node is descendent:
                return index
            if node is None:
                next_nodes.append(None)
                next_nodes.append(None)
            else:
                if node.left is not None and node.right is not None:
                    has_more_node = True
                next_nodes.append(node.left)
                next_nodes.append(node.right)
            index += 1
        current_nodes = next_nodes
    raise Exception("Given node not found in the tree")
                

def get_parent(root: Optional[Node], child: Optional[Node]) -> Optional[Node]:
    if child is None:
        return None
    stack: List[Optional[Node]] = [root]
    while stack:
        node = stack.pop()
        if node:
            if node.left is child or node.right is child:
                return node
            stack.append(node.left)
            stack.append(node.right)
    return None
    
def _build_bst_from_sorted_values(sorted_values: List[int]) -> Optional["Node"]:
    if len(sorted_values) == 0:
        return None        
    mid = len(sorted_values) // 2
    root = Node(sorted_values[mid])
    root.left = _build_bst_from_sorted_values(sorted_values[:mid-1])
    root.right = _build_bst_from_sorted_values(sorted_values[mid+1:])
    return root

In [90]:
arr1 = [2, 5, None, 3, None, 1, 4]
arr2 = [2, None, 4, None, 3, None, 1, 4]
arr3 = [0, 1, 1, 2, 3, 3, 2]
arr4 = list(range(10))
bst_arr = [5, 3, 7, 2, 4, 6, 8]
root1 = build2(arr1)
root2 = build2(arr2)
root3 = build(arr3)
root4 = build(arr4)
root_bst = build(bst_arr)

In [93]:
root3.properties

{'height': 2,
 'size': 7,
 'is_max_heap': False,
 'is_min_heap': True,
 'is_perfect': True,
 'is_complete': True,
 'is_full': True,
 'leaves_count': 4,
 'min_node_val': 0,
 'max_node_val': 3,
 'min_leaf_depth': 2,
 'max_leaf_depth': 2,
 'is_balanced': True,
 'is_bst': False,
 'is_symmetric': True}